In [1]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR, SVC
from dateutil import parser

In [2]:
scores_df = pd.read_csv("LW_Manual_scores_for_ICC_2020-05-20.csv")

#hit scores
hit_df = scores_df[["subject","date","crossing number","run","limb","human_hit_1","human_hit_2","human_hit_3"]]
#take the mode. If no mode, take the mean and round up
hit_df.loc[:,"human_hit"]=hit_df[["human_hit_1","human_hit_2","human_hit_3"]].mode(axis=1).mean(axis=1).round(0)

#score extraction
dom_f_hit = hit_df.loc[hit_df["limb"]=="Dominant Front"]


/home/isabelle/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/isabelle/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [3]:
rat_folder = glob.glob("*/dlc_output_resnet50/*.h5")
rung_folder = glob.glob("*/dlc_rung_r50/*.h5")

In [5]:
X = []
y = []
lengths = []
for file in rat_folder:
    #rat tracking file
    #name of the rung file
    rung_name_list = file.split("/")[-1].split("_")[0:8]+["LadderWalkMar12shuffle1","450000.h5"]
    rung_file = '_'.join(rung_name_list)
    #open the file
    rat_df = pd.read_hdf(file)["DLC_resnet50_LadderWalkFeb13shuffle1_450000"]
    #properties of the file
    subject = file.split("/")[0]
    date_raw = rung_name_list[1]
    date = parser.parse(date_raw).date().strftime("%Y-%m-%d")
    run = rung_name_list[2]
    crossing = [int(s) for s in rung_name_list[3] if s.isdigit()][0]
    #
    dom_f_hit_score = dom_f_hit[(dom_f_hit["subject"]==subject) & (dom_f_hit["date"]==date) & (dom_f_hit["run"]==run)].reset_index()
    if len(dom_f_hit_score) == 0:
        print(subject + date + run + "not found")
        continue
    
    
    #open corresponding rung file
    #rung_df = pd.read_hdf(subject+"/dlc_rung_r50/"+rung_file)["DLC_resnet50_LadderWalkMar12shuffle1_450000"]

    #join the rat and rung dataframes
    #merge_df = rat_df.join(rung_df)
    dom_f_cols = ['right elbow','right wrist','right fingers']

    #new dataframe with columns removed
    #dom_f = merge_df[dom_f_cols_up]
    dom_f_temp = rat_df[dom_f_cols]
    
    #remove where median likelihood is low
    dom_f_low_like = dom_f_temp.columns[-dom_f_temp[dom_f_temp.columns.get_level_values(0).unique()].median().ge(0.2)].get_level_values(0).tolist()
    dom_f_cols_up = [x for x in dom_f_cols if x not in dom_f_low_like ]

    dom_f = rat_df[dom_f_cols_up]
    if dom_f.shape[1] == 0:
        continue
    y.append(dom_f_hit_score["human_hit"][0])
    #if len(dom_f_low_like) >0:
        #print(dom_f_low_like)
    dom_f = dom_f.drop('likelihood', axis=1, level=1)

    #scale data
    ####Uses the values function of pandas - converts any dataframe to an array
    data_for_scaling = dom_f.values
    #### the scaling object
    scaler = MinMaxScaler()
    #We will use fit_transform here - we want to actually scale this data, not use the scaler on a different dataset
    scaled_data = scaler.fit_transform(data_for_scaling)
    #PCA
    #data_reducer = PCA(n_components=5)
    #pca_data = data_reducer.fit_transform(scaled_data)
    ''' This loop gives us the top feature of each component using the argmax function '''
    #top_features = [np.abs(data_reducer.components_[i]).argmax() for i in range(data_reducer.components_.shape[0])]


    ''' Now, we go back to our original feature names (columns of our dataset), and get the names of the 4 key features '''
    #top_feature_names = [list(dom_f.columns)[top_features[i]] for i in range(data_reducer.components_.shape[0])]

    ''' printing the names '''
    #for n,name in enumerate(top_feature_names):
        #print("#" + str(n) + ": " + str(name))

    #print(data_reducer.components_)
    #print(data_reducer.explained_variance_ratio_)
    
    
    if len(scaled_data) < 356:
        newlength = (356-len(scaled_data))
        zero = np.zeros((newlength,6))
        scaled_temp = pd.DataFrame(scaled_data).append(pd.DataFrame(zero),ignore_index=True)
        scaled_temp = scaled_temp.fillna(0)
        scaled_data2 = scaled_temp.values
    else:
        scaled_data2 = scaled_data
    
    video_data = scaled_data2.flatten()
    lengths.append([len(video_data)])
    X.append(video_data)
    
    

MC702019-03-14L4not found
MC702019-04-09L4not found


In [6]:
np.unique(lengths)

array([2136])

In [7]:
len(X),len(y)

(230, 230)

In [8]:
regressor = SVR(kernel="rbf")
regressor.fit(X,y)
y_pred = regressor.predict(X)

In [9]:
y-y_pred.round(0)

array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.,
        0.,  0.,  0., -1., -1.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,
        0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  1.,  1., -1.,  0.,
       -1.,  0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  1.,  0.,  1.,  0.,
        0., -1.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -1., -1., -1.,  0.,
        1., -1.,  0.,  0.,  1.,  0., -1.,  0.,  0., -2.,  0.,  0., -1.,
        1.,  0.,  0.,  0., -1.,  0.,  1., -1.,  1., -1.,  0., -1.,  1.,
        1., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0., -1.,
       -1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0., -1.,  0.,  0.,
       -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  1.,
        1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0

In [10]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y,y_pred.round(0),squared=True)

0.5260869565217391

In [12]:
regressor = SVC(kernel="rbf")
regressor.fit(X,y)
y_pred = regressor.predict(X)

In [13]:
y-y_pred

array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
       -1.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.,
        0.,  1.,  0., -1.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  0.,  0.,
        0., -1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  1.,  1., -2.,  0.,
       -2.,  0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  1.,  0.,  1.,  0.,
        0., -1.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -1., -1., -1.,  0.,
        1., -1.,  0.,  1.,  1.,  0., -1.,  0.,  0., -2.,  0.,  0., -1.,
        1.,  0.,  0.,  0., -1.,  0.,  1., -1.,  1., -2.,  0., -1.,  1.,
        1., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1., -1.,
       -1.,  0., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., -1.,  0.,  0.,
       -1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  1.,  1.,  0., -1.,  1.,
        1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  0., -1.,  1.,  0

In [14]:
mean_squared_error(y,y_pred.round(0),squared=True)

0.7478260869565218

In [15]:
set(y)

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0}